# 4. ZPVE Correction and Final Binding Energy Distribution

At this point we have submitted all necessary computations to construct a binding energy distribution on the 
ice-mantle model comprising five W12 cluster. So now we can proceed to put it all together and build the final binding energy distribution!  

First we need to process the Hessian matrices to calculate the ZPVE. For this purpose we will use a modified version of the Psi4 library for harmonic analysis.  Therefore to be able to use it we need to install Psi4 first on the computer where you will be running this jupyter notebook (Note that this process does not require much processing power. All the heavy lifting has already been done!). Instructions on how to install Psi4 using conda can be found on the [Psi4 webpage](https://psicode.org/installs/v15/). So first we will load Psi4 as a python module and also import the modifed vibrational analysis function from beep.

In [1]:
import psi4
from beep.be_tools import zpve_correction

First we need to call the client and  the name of the `ReactionDataset` that contains the stoichiometry information and the binding energy values. Since we computed the Hessian at the HF-3C/Minix geometry we need to specify that in the name of the `ReactionDataset`

In [2]:
import qcfractal.interface as ptl
client = ptl.FractalClient(address="localhost:7777", verify=False, username='', password='')

molecule  = "ch3oh"
cluster = "4"
name_opt = molecule+'_W12_'+ cluster
name_be = 'be_'+name_opt+'_hf3c'

opt_method = 'hf3c_minix'
be_method = 'B3LYP-D3BJ'

Now we can retrieve the ZPVE correction using the `zpve_correction` method contained in be_tools. It returns 
a Pandas DataFrame with the non corrected, ZPVE corrected binding energies and the ZPVE correction, a list with linear fit parameters and a plot with the linear fit parameters. 

In [ ]:
df_zpve_corr, fit_data, fig =  zpve_correction(name_be, be_method, opt_method, client=client)

Next we  in concatenate the DataFrames containing the binding energies bound to the 5 water clusters and correct the values with the linear model we just obtained. 

In [ ]:
# Querying the database and storing binding energy results in a pandas dataframe (df_be): 

ds_be_list =[]

for i in range(1,6):
    ds_be = client.get_collection("ReactionDataset", 'be_'+ molecule +'_W12_'+"%02d" %i+'_'+lot_opt[molecule].split("_")[0])  
    val = ds_be.get_values(stoich='default', method=be_method)
    ds_be_list.append(val)
    print("Frame {}: Structures added to the E_b distribution: {} ".format(str(i), len(val)))

df_be = pd.concat(ds_be_list)
print("Total number of structures in the E_b distribution: {} ".format(len(df_be)))

Finally we have a BE distribution  that we can convert  to Kelvin using QCElemental conversion factors and  visualize the resulting distribution as a histogram:

In [ ]:
import qcelemental as qcel

v_K = -be_values * qcel.constants.conversion_factor("kcal/mol", 'K')
v_K = v_K[v_K > 0]

nbins = 12
fig = plt.figure(figsize=(8,4))
plt.hist(v_K, bins=nbins)
plt.xlim([0, 7000])
plt.xlabel('$E_b$ / $K$ ', size = 15)
plt.ylabel('frequency number',size = 15)